# Metrics using the main Moments (or Covariance Matrix)

## Loss Function 
Label $y$: $$y = \text{y_true} \in \mathbb{R}^{n\times1} \quad \text{, with} \quad n = 7 = \texttt{NUM_OUT_DIM}$$

Network-Output $o$: $$o = \text{pred_vects} \in \mathbb{R}^{m\times 7} \quad \text{, with} \quad m = \texttt{NUM_OUT_CHANNELS}$$

$$o_{ij} = \left(t_i\right)_j \quad\text{ or }o = \begin{pmatrix}
t_0\\ t_1\\\vdots\\ t_m\end{pmatrix}, t_i \in \mathbb{R}^{1\times7}$$

Reinterpretation of the Network Output (different proposed paths $t_i$) as "Covariance Matrix" aka Matrix of Main-Moments $\Sigma$: $$ \Sigma = \epsilon I_n + \frac{1}{m}\sum\limits_{i}^m t_i^T\cdot t_i      \quad \text{, with}\quad I_n\text{ as Identity-Matrix}$$

Loss $l$: $$\begin{align}
l &= -\ln p\left(y|x;\Sigma\right)\\
\end{align}$$

Multidimensional Gaussian Distribution: $$
\begin{align}
p(y) &= (2\pi)^{-\frac{n}{2}} \cdot \left|\Sigma\right|^{-\frac{1}{2}} \cdot \exp\left(-\frac{1}{2}\left(y-\mu\right)^T\cdot\Sigma^{-1}\cdot \left(y-\mu\right)\right) \\
\Rightarrow l &=  -\left( \underbrace{-\left(\frac{n}{2}\right)\cdot\ln\left(2\pi\right)  -\frac{1}{2}\ln\left|\Sigma\right|}_{\text{previously scaling factor}}  \underbrace{-\frac{1}{2}y^T \Sigma^{-1} y}_{\text{previously exponent}}         \right)\\
&= \frac{1}{2}\left(\underbrace{n \cdot\ln\left(2\pi\right)}_{\text{constant offset}}  + \ln\left|\Sigma\right| + y^T \Sigma^{-1} y \right)
\end{align}$$

With the assumption: $\mu =0$, so that the network does not have a preference to drive ie. forwards without input.

### Formating in Code
`label (y_true)` is a (flat) row Tensor of shape: $\begin{pmatrix}t_0,t_1,\dots,t_n\end{pmatrix}$, **not** $\begin{pmatrix}
t_0\\t_1\\\vdots\\t_n\end{pmatrix}$ : ``'tf.Tensor([...], shape=(7,), dtype=float32)'``

`network-output (pred_vects)` is a tensorflow Tensor of shape (batch, m, 7), therefore also a stack of (flat) row Tensors

Adjusted loss (without constant offset $\left(n\cdot\ln\left(2\pi\right)\right)$ and factors $\left(\frac{1}{2}\right)$) used in code:

$$ \hat{l} =   \ln\left|\Sigma\right| + y^T \Sigma^{-1} y          $$

In [ ]:
import tensorflow as tf
tf.config.experimental.set_memory_growth((tf.config.list_physical_devices('GPU'))[0], True)
from math import pi
import os

In [ ]:
@tf.function
def getSigma(pred_vects):
    """Calculates Sigma (Covariance Matrix) based on the network output pred_vects
    Format pred_vects: (batch_size x m x n) , with n=number of dimensions
    Calculation: Sum of outer Products of every set of dimensions with itself and divide by number of sets to normalise . (and add epsilon)
    Method: for each batch: pred_vects^T    @     pred_vects
                            {shape n x m} matmul {shape n x m}
    Returns sigma of shape (batch_size x n x n)
    
    Adds an epsilon of tf.keras.backend.epsilon() (*Identity Matrix) to ensure Invertabillity
    """
    sigma = tf.matmul(pred_vects, pred_vects, transpose_a=True) 
    sigma = sigma / pred_vects.shape[-2] # Divide by m to normalise sigma invariant of number of input vectors
    # Add epsilon to sigma to avoid zeros
    sigma = tf.add(sigma, tf.eye(sigma.shape[-1], batch_shape=sigma.shape[:0]) * tf.keras.backend.epsilon())
    return sigma

@tf.function
def momentProb(y_true, pred_vects):
    """Returns the probability of the label y_true given the output of the network pred_vects
    
    Format y_true: (batch_size x n) , with n=numer of dimensions
    Format pred_vects: (batch_size x m x n) , with n=number of dimensions
    
    Returns probability per batch as (batch_size)
    
    Probability = (2 pi)^(-n/2) * 1/sqrt(det(Sigma)) * exp(-1/2 y^T * Sigma^(-1) * y)
    """
    sigma = getSigma(pred_vects)
    
    scalingFactor = (2*pi)**(-(sigma.shape[-1])/2) 
    det_part = tf.math.reciprocal_no_nan(tf.sqrt(tf.linalg.det(sigma)))
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = -1/2 *  tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y))
    return scalingFactor * det_part * tf.exp(exponent)

@tf.function
def momentLoss(y_true, pred_vects):
    """Calculates the loss of the label value y_true based on the Gaussian Distribution defined by the network output (pred_vects)
    Uses the probability based on Multidimensional Gaussian Distribution, but removes the standard offset of n*ln(2pi) and the factor 1/2 

    Format y_true: (batch_size x n) , with n=numer of dimensions
    Format pred_vects: (batch_size x m x n) , with n=number of dimensions
    
    Returns loss per batch as (batch_size x 1)
    """
    sigma = getSigma(pred_vects)
    det_part = tf.linalg.logdet(sigma) # = log(det(sigma))
    #det_part = tf.math.log(tf.linalg.det(sigma)+tf.keras.backend.epsilon()) # = log(det(sigma))
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y)) 
    return det_part + exponent

@tf.function
def momentLoss_onlyExponent(y_true, pred_vects):
    """Equal to Momentloss, but without the determinant part"""
    sigma = getSigma(pred_vects)
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y)) 
    return exponent

@tf.function
def momentLoss_onlyExponent_onSigma(y_true, sigma):
    """Equal to Momentloss, but without the determinant part and based on sigma"""
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y)) 
    return exponent

@tf.function
def momentLoss_onSigma(y_true, sigma):
    """Equal to momentLoss, but with given sigma"""
    sigma = tf.add(sigma, tf.eye(sigma.shape[-1], batch_shape=sigma.shape[:0]) * tf.keras.backend.epsilon())
    det_part = tf.linalg.logdet(sigma) # = log(det(sigma))
    #det_part = tf.math.log(tf.linalg.det(sigma)+tf.keras.backend.epsilon()) # = log(det(sigma))
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y)) 
    return det_part + exponent
    
@tf.function
def momentLoss2(y_true, pred_vects):
    """Slower variant of momentLoss, but with mathematical correcter result (= -ln(prob) )
    
    Calculates the loss of the label value y_true based on the Gaussian Distribution defined by the network output (pred_vects)
    Uses the probability based on Multidimensional Gaussian Distribution, without removing the standard offset of n*ln(2pi) and the factor 1/2 
    
    Format y_true: (batch_size x n) , with n=numer of dimensions
    Format pred_vects: (batch_size x m x n) , with n=number of dimensions
    
    Returns loss per batch as (batch_size x 1)
    """
    sigma = getSigma(pred_vects)
    const_offset = sigma.shape[-1] * tf.math.log(2 * pi)
    det_part = tf.math.log(tf.linalg.det(sigma))
    inv_sigma = tf.linalg.inv(sigma)
    y = tf.expand_dims(y_true, axis=-1) # necessary to enable matmul to work with batch x n x 1, undone with squeeze in next line
    exponent = tf.squeeze(tf.matmul(tf.matmul(y, inv_sigma, transpose_a=True), y)) 
    return 1/2 * (const_offset + det_part + exponent)

@tf.function
def getEigen(pred_vects):
    """Returns the Eigenvalues and Eigenvectors defined by the network output pred_vects
    Format: [eigenvalues, eigenvectors], in the order of eigenvalues by tf.linalg.eigh
    
    Warning: eigenvectors is an square matrix. Each Vector is in a column. Therefore
     you CANNOT iterate through eigenvectors with i in eigenVectors. Use Transpose!
    """
    sigma = getSigma(pred_vects)
    return tf.linalg.eigh(sigma)

# Print and Debug

In [ ]:
#%matplotlib notebook
import matplotlib.pyplot as plt, numpy as np
def plotOutput(pred_vects, figsize=(20,10), dimension_names=["X","Y","Z","Rot_X","Rot_Y","Rot_Z","Finger"],
              cmap ="gray", showPure=True, showSigma=True, showEigen=True, storeImages=None, plotProbTrue=None):
    """Plots the network output pred_vects of a single prediction as images
    param figsize: 2-Dim Shape defining the figure size of the Eigenvector Plot
    param dimension_names: list of dimension names. Must be of same length as pred_vects[-1]
    param cmap: cmap parameter to be passed on to matpotlib
    param showPure: weather to handle the pure output
    param showSigma: weather to handle the Sigma Covariance matrix
    param showEigen: weather to handle the Eigenvectors
    param storeImages: Path to store the handled images to
    param plotProbTrue: true value to plot. Has to be set to a vector of same size as eigenvektor
    
    """
    assert tf.rank(pred_vects) == 2, "plotOutput requires output of single prediction, not batch."
    sigma = getSigma(pred_vects)
    ndims = sigma.shape[-1]
    if showPure:
        # Plot network output
        plt.figure(figsize=figsize)
        plt.suptitle("Output Vectors")
        for i,v in enumerate(pred_vects):
            plt.subplot(1,len(pred_vects),1+i)
            plt.imshow(abs(np.expand_dims(v, axis=1)), cmap=cmap)
            plt.xticks([])
            plt.yticks(range(ndims), dimension_names)
            for j, vi in enumerate(v):
                plt.text(0,j,f"{vi:.1f}", horizontalalignment='center', verticalalignment='center')
        plt.tight_layout()
        if storeImages:
            plt.savefig(os.path.join(storeImages,"output.png"))
            plt.close()
        else:
            plt.show()
    
    if showSigma:
        # Plot Sigma
        plt.figure()
        plt.title("Output Sigma $\Sigma$")
        plt.imshow(abs(sigma.numpy()), cmap=cmap)
        plt.yticks(range(ndims), dimension_names)
        plt.xticks(range(ndims), dimension_names)
        plt.tight_layout()
        if storeImages:
            plt.savefig(os.path.join(storeImages,"covariance.png"))
            plt.close()
        else:
            plt.show()
    if showEigen:
        # Plot Eigenvectors and values in descending order
        vals, vecs = tf.linalg.eigh(sigma) 
        plt.figure(figsize=figsize)
        plt.suptitle("Eigenvalues and -vectors")
        for i, (val, vec) in enumerate(zip(vals, tf.transpose(vecs))):
            plt.subplot(1,ndims,ndims-i)
            plt.title(f"$\lambda=${val.numpy():.2f}")
            plt.imshow(abs(np.expand_dims(vec.numpy(), axis=1)), cmap=cmap)
            plt.xticks([])
            plt.yticks(range(ndims), dimension_names)
            plt.xlabel(f"loss: {momentLoss(vec, pred_vects):.2f}")
            for j, v in enumerate(vec):
                plt.text(0,j,f"{v:.1f}", horizontalalignment='center', verticalalignment='center')
        plt.tight_layout()
        if storeImages:
            plt.savefig(os.path.join(storeImages,"eigenvectors.png"))
            plt.close()
        else:
            plt.show()
    if not plotProbTrue == None:
        plt.plot(plotProbTrue)
        plt.imshow(abs(np.expand_dims(plotProbTrue, axis=1)), cmap=cmap)
        plt.xticks([])
        plt.yticks(range(ndims), dimension_names)
        plt.xlabel(f"loss: {momentLoss(plotProbTrue, pred_vects):.2f}")
        for j, v in enumerate(plotProbTrue):
            plt.text(0,j,f"{v:.1f}", horizontalalignment='center', verticalalignment='center')
        plt.show()

## keras prerequisites:
- requests (assumes during compiling) labels and network-output to have the same rank
- network output has shape `( batch_size, ... )`

In [ ]:
if __name__=="__main__":
    # Test metrics
    out = [[
           [4,0,0],
           [0,2,0],
           [0,1,0],
           [0,1,0],
          ],[
           [5,0,0],
           [0,3,0],
           [0,1,0],
           [0,1,0],
          ]]
    out = tf.convert_to_tensor(out,dtype=tf.dtypes.float32)
    
    print(f"Network shape: {out.shape}: (batch-size x m_vectors x n_dims)")
    print(f"Output:")
    for o in out:
        plotOutput(o, figsize=(4,2), dimension_names=["X","Y","Z"])
        break
    def printlabel(label):
        label= tf.convert_to_tensor(label,dtype=tf.dtypes.float32)
        label = tf.math.l2_normalize(label, axis=1)
        loss, prob, loss2 = momentLoss(label, out), momentProb(label,out) , momentLoss2(label, out)
        print(f"label: {label}\n    Loss    : {loss}\n    Prob    : {prob}\n    Loss2   : {loss2}\n    ln(prob): {-tf.math.log(prob)}")
    print("--- TRIVIAL CASES: ---")
    printlabel([[1,0,0],[0,1,0]])
    printlabel([[0,1,0],[0,1,0]])
    print("\n-- Should cause Errors (be less probable) -- ")
    printlabel([[0,0,1],[0,0,1]])
    printlabel([[0,1,1],[0,1,1]])
    printlabel([[1,1,1],[1,1,1]])